In [ ]:
import cv2
from tensorflow.python.keras.models import model_from_json
import numpy as np
import tensorflow as tf
tf.compat.v1.disable_eager_execution()  

# Load the model architecture from JSON file
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()

# Create model from JSON
model = model_from_json(model_json)

# Load model weights from the .h5 file
model.load_weights("facialemotionmodel.h5")

# Path to Haar cascade XML file
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Function to extract features from an image
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Access the webcam
webcam = cv2.VideoCapture(0)

# Labels for emotion classes
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

while True:
    # Read frame from webcam
    i, im = webcam.read()
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(im, 1.3, 5)
    
    try:
        # Iterate through detected faces
        for (p, q, r, s) in faces:
            # Extract face region
            image = gray[q:q+s, p:p+r]
            cv2.rectangle(im, (p, q), (p+r, q+s), (255, 0, 0), 2)
            
            # Resize face image to match model input shape
            image = cv2.resize(image, (48, 48))
            
            # Extract features and preprocess
            img = extract_features(image)
            
            # Make prediction
            pred = model.predict(img)
            prediction_label = labels[pred.argmax()]
            
            # Display predicted emotion label
            cv2.putText(im, '% s' %(prediction_label), (p-10, q-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))
        
        # Display output frame
        cv2.imshow("Output", im)
        
        # Check for key press to exit (ESC key)
        key = cv2.waitKey(1) 
        if key == 27:  # ESC key
            break
    except cv2.error:
        pass

# Release webcam and close OpenCV windows
webcam.release()
cv2.destroyAllWindows()


C:\Users\ADMIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\keras\engine\training.py:2455: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
